In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


### Batch for Get Melodic

In [5]:
# Example passing some parameters to `func` calls. Note that you only add
# the parameters to kwargs that you need to pass. This example returns a
# list of dataframes of the melodic intervals of each piece in the corpus,
# and in this case will be chromatic and undirected intervals because of
# parameters passed in kwargs.


corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                     'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func = ImportedPiece.getMelodic  # <- NB there are no parentheses here
kwargs = {'kind': 'c', 'directed': False} 
list_of_dfs = corpus.batch(func, kwargs, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df.fillna('-')

Memoized piece detected.
Memoized piece detected.


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,Rest,-,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,-
1,Rest,0,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,-
2,-,2,-,-,Antoine de Févin,Missa Mente tota: Credo,-
3,-,2,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,-
4,-,2,-,-,Antoine de Févin,Missa Mente tota: Credo,-
...,...,...,...,...,...,...,...
1319,-,-,2,-,"Cadéac, Pierre",Je suis déshéritée,2
1320,-,3,1,3,"Cadéac, Pierre",Je suis déshéritée,0
1321,-,-,2,-,"Cadéac, Pierre",Je suis déshéritée,-
1322,-,2,2,1,"Cadéac, Pierre",Je suis déshéritée,-


### Here is Batch for Melodic NGrams

In [6]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func1 = ImportedPiece.getMelodic
# NB: you would probably want to set metadata to False for most preliminary results
list_of_dfs = corpus.batch(func=func1, kwargs={'end': False}, metadata=True)
func2 = ImportedPiece.getNgrams
list_of_melodic_ngrams = corpus.batch(func=func2, kwargs={'n': 4, 'df': list_of_dfs})
combined_df = pd.concat(list_of_melodic_ngrams, ignore_index=True)
combined_df.fillna('-')

Memoized piece detected.
Memoized piece detected.


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,-,"P1, -M2, M2, -M2",-,-,Antoine de Févin,Missa Mente tota: Credo,-
1,-,"-M2, M2, -M2, M2",-,-,Antoine de Févin,Missa Mente tota: Credo,-
2,-,"M2, -M2, M2, M2",-,-,Antoine de Févin,Missa Mente tota: Credo,-
3,"P1, -M2, M2, -M2","-M2, M2, M2, m2",-,-,Antoine de Févin,Missa Mente tota: Credo,-
4,-,"M2, M2, m2, -m2",-,-,Antoine de Févin,Missa Mente tota: Credo,-
...,...,...,...,...,...,...,...
1355,-,"M2, m2, M2, -M2","m2, M2, -M2, -m2",-,"Cadéac, Pierre",Je suis déshéritée,-
1356,-,"m2, M2, -M2, -m3","M2, -M2, -m2, -M2",-,"Cadéac, Pierre",Je suis déshéritée,-
1357,-,"M2, -M2, -m3, M2",-,-,"Cadéac, Pierre",Je suis déshéritée,-
1358,-,-,"-M2, -m2, -M2, M2",-,"Cadéac, Pierre",Je suis déshéritée,-


### Here is Batch for Cadences

In [25]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
piece_list = ['CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']
corpus_list = []
for work in piece_list:
    url = git_prefix + work
    corpus_list.append(url)
# Example using batch to count the cadence types from multiple pieces:
corpus = CorpusBase(corpus_list)
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
combined_df = combined_df[col_list]
combined_df
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
# Get the number of cadences per Beat level:
cadTypeCounts = combined_df['Beat'].value_counts()

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


In [29]:
combined_df.to_csv("saved_csv/Mass_0019_Cads.csv")

In [27]:
combined_df['all_features'] = combined_df['CadType'] + "_" + combined_df['Tone'] + "_" + combined_df['Evaded'].apply(str)

In [28]:
combined_df

,Composer,Title,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext,all_features
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,7,1.0,NaN,G,NaN,NaN,G3,P1,P1,0.090909,48.0,36.0,NaN
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,11,3.0,Clausula Vera,G,True,1.0,G3,P1,P1,0.159091,36.0,52.0,Clausula Vera_G_True
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,18,1.0,Authentic,G,True,1.0,B3,M3,P1,0.257576,52.0,16.0,Authentic_G_True
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,20,1.0,Authentic,G,False,1.0,G3,P1,P1,0.287879,16.0,100.0,Authentic_G_False
4,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,32,3.0,Clausula Vera,C,True,1.0,C4,P4,-P5,0.477273,100.0,44.0,Clausula Vera_C_True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,62,3.0,Clausula Vera,D,False,0.0,D3,M2,M2,0.880282,4.0,24.0,Clausula Vera_D_False
110,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,65,3.0,Clausula Vera,D,False,0.0,D3,M2,M2,0.922535,24.0,8.0,Clausula Vera_D_False
111,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,66,3.0,Authentic,D,False,0.0,D3,M2,M2,0.936620,8.0,12.0,Authentic_D_False
112,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,68,1.0,Authentic,G,False,1.0,G3,P5,P5,0.957746,12.0,2.0,Authentic_G_False
